<a href="https://colab.research.google.com/github/hanovec/bmc_2/blob/main/Business_Model_Canvas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: Setup, Configuration, and Initialization

# Library installations
!pip install -q google-generativeai
!pip install -q python-dotenv

import os
import time
import google.generativeai as genai
import json
import IPython

# --- Configuration Section ---

# Model Configuration
# A prioritized list of model name "stems" to search for.
# The script will use the first one it finds available.
# CORRECTED: Your specific model is now the top priority.
PRIORITY_MODEL_STEMS = [
    "gemini-2.5-flash-preview-05-20",  # Your specifically requested preview model.
    "gemini-1.5-flash-latest",          # Fallback 1: A stable, recent Flash model
    "gemini-1.5-pro-latest",            # Fallback 2: A recent, powerful Pro model
    "gemini-pro",                       # Fallback 3: A widely available Pro model
]

# Generation & Safety Configuration
GENERATION_CONFIG = {
    "temperature": 1.5,
    "top_p": 0.95,
    "max_output_tokens": 65536,
}

SAFETY_SETTINGS = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
]
# --- End of Configuration ---


# --- Authentication and Initialization ---
model = None # Initialize model to None to ensure it exists
print("Starting API Key Configuration...")
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    print("API Key successfully loaded from Google Colab Secrets.")
except ImportError:
    print("Not in Google Colab. Attempting to load API Key from environment variable...")
    from dotenv import load_dotenv
    load_dotenv()
    GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

if not GOOGLE_API_KEY:
    raise ValueError("Google API Key not found. Please set it up in Colab Secrets or as an environment variable.")
else:
    genai.configure(api_key=GOOGLE_API_KEY)
    print("Google Generative AI API configured.")

try:
    print("\nSearching for an available Gemini model...")
    model_name_to_use = None
    available_models = [m for m in genai.list_models() if "generateContent" in m.supported_generation_methods]

    for model_stem in PRIORITY_MODEL_STEMS:
        # Find the first available model that contains the priority stem and isn't a vision model
        found_model = next((m for m in available_models if model_stem in m.name and 'vision' not in m.name.lower()), None)
        if found_model:
            model_name_to_use = found_model.name
            print(f"  > Found priority model: {model_name_to_use}")
            break

    if not model_name_to_use:
        raise ValueError("Could not find any of the priority models. Please check available models in your region and ensure your project has access to any requested preview models.")

    model = genai.GenerativeModel(
        model_name=model_name_to_use,
        generation_config=GENERATION_CONFIG,
        safety_settings=SAFETY_SETTINGS
    )
    print(f"Model '{model_name_to_use}' initialized successfully.")

except Exception as e:
    print(f"CRITICAL ERROR during model initialization: {e}")
    print("Please check your API key, model availability in your region, and project quotas.")
    # The 'model' variable will remain None, which will be caught by checks in other cells.

Starting API Key Configuration...
API Key successfully loaded from Google Colab Secrets.
Google Generative AI API configured.

Searching for an available Gemini model...
  > Found priority model: models/gemini-2.5-flash-preview-05-20
Model 'models/gemini-2.5-flash-preview-05-20' initialized successfully.


In [ ]:
# Cell 2: Helper Function to Interact with Gemini (Simplified)

def ask_gemini_sdk(prompt_text: str, temperature: float = None) -> str:
    """
    Sends a prompt to the initialized Gemini model and returns the text response.
    Allows for correctly overriding the default temperature for a specific call.
    """
    if not model:
        return "AI_ERROR: Model not initialized. Please ensure Cell 1 executed correctly."

    config_overrides = {}
    if temperature is not None:
        config_overrides['temperature'] = float(temperature)
        display_status_message(f"AI is thinking with custom temperature: {config_overrides['temperature']}...")
    else:
        display_status_message("AI is thinking with default temperature...")

    try:
        # The prompt_content is now just the text, no files.
        response = model.generate_content(prompt_text, generation_config=config_overrides)

        if response.parts:
            return response.text.strip()
        elif response.prompt_feedback and response.prompt_feedback.block_reason:
            reason = response.prompt_feedback.block_reason.name
            return f"AI_ERROR: Your prompt was blocked for safety reasons ({reason}). Please rephrase your input."
        else:
            # Handle other cases like unexpected stops or empty responses
            return "AI_ERROR: Received an incomplete response from the model. Please try again."
    except Exception as e:
        display_status_message(f"ERROR during API call: {e}")
        return f"AI_ERROR: An unexpected error occurred: {type(e).__name__}. Please check console."

In [ ]:
# Cell 3: Phase 1 - Welcome and Introduction (Streamlined)

def display_welcome_and_introduction():
    """
    Displays a single, comprehensive welcome message using a styled box without any pauses.
    """
    welcome_text = (
        "Welcome to the BMC Navigator! I'm your AI business coach, here to help you map and "
        "innovate your IT business model using the powerful Business Model Canvas (BMC). "
        "We will begin by uploading your methodology document(s), which will guide our entire analysis."
    )
    ai_box(welcome_text, title="🚀 Welcome Aboard!")

In [ ]:
# Cell 4: Phase 2 - Input Gathering and Validation (with Comprehensive Sub-points Display)

# LLM_BMC_HELPER_PERSONA prompt remains the same, it's used for short clarifications.

def get_user_input_with_llm_validation(bmc_block_question: str, block_name: str, coverage_points: list = None, it_examples: list[str] = None) -> str:
    """
    Prompts the user with a main question and a checklist of points to cover.
    Uses styled boxes and Gemini for clarification if the input is vague.
    """
    # IMPROVEMENT: Build a rich HTML prompt that includes the coverage points as a checklist.
    full_prompt_html = f"<p style='font-size: 1.1em; margin-bottom: 15px;'>{bmc_block_question}</p>"

    if coverage_points:
        full_prompt_html += "<p style='margin-bottom: 5px; font-weight: bold;'>Pro komplexní odpověď zvažte prosím následující body:</p>"
        full_prompt_html += "<ul style='margin-top: 5px; margin-left: 20px;'>"
        for point in coverage_points:
            full_prompt_html += f"<li style='margin-bottom: 5px;'>{point}</li>"
        full_prompt_html += "</ul>"

    if it_examples:
        full_prompt_html += f"<p style='margin-top: 15px; font-style: italic; color: #555;'>Například: {', '.join(it_examples)}.</p>"


    while True:
        # Step 1: Ask the user for input using the rich HTML prompt
        user_response = user_prompt_box(full_prompt_html)
        user_response_stripped = user_response.strip()

        # Step 2: Immediately redisplay their answer
        display_user_response(user_response_stripped)

        # Step 3: Validation logic (remains the same)
        if user_response_stripped.lower() in ["n/a", "none", "skip"]:
            ai_box(f"Understood. We'll skip '{block_name}' for now.", title="✅ Acknowledged")
            return "Skipped"

        if len(user_response_stripped) < 25: # Slightly increased threshold for comprehensive answers
            ai_box(f"Děkuji. To je dobrý začátek, ale zkusme přidat více detailů k jednotlivým bodům.", title=" digging deeper...")
            # The clarification logic can remain simple, as the user already has the detailed prompt.
            # No need to call the LLM again here unless absolutely necessary.
        else:
            return user_response_stripped

In [ ]:
# Cell 5: Phase 2 - AI-Driven Question Plan Generation

# IMPROVEMENT: The prompt now instructs the AI to use its own deep knowledge of the standard BMC methodology.
LLM_EXPERT_QUESTION_PLANNER = """
You are an expert strategy consultant and a master of the Alex Osterwalder Business Model Canvas methodology. Your task is to create a structured, comprehensive questioning plan to guide a user through a deep-dive description of their IT Reseller/System Integrator business.

Your output MUST be a valid JSON list of 9 objects, one for each core block of the Business Model Canvas. The order should be logical (Customers/Value -> Operations -> Finances). Each object must have the following four keys:
1. "key": The standard snake_case identifier for the block (e.g., "customer_segments").
2. "question": The main, user-friendly question for the block.
3. "coverage_points": A list of 3-4 critical sub-questions or topics the user MUST consider to provide a complete answer for that block. These should be insightful and cover the nuances of the methodology.
4. "examples": A list of 3-4 short, relevant examples for an IT Reseller/System Integrator.

Example of one object in the list:
{
  "key": "value_propositions",
  "question": "Now, let's detail your Value Propositions. What value do you deliver to your customers?",
  "coverage_points": [
    "Which specific customer problem are you solving or which need are you satisfying?",
    "What bundle of products and services are you offering to each segment?",
    "How does your offering differ from competitors (e.g., is it about performance, price, design, convenience)?",
    "Are you offering something new and disruptive, or improving an existing solution?"
  ],
  "examples": ["Managed Cybersecurity (SOC-as-a-Service)", "Custom Cloud Migration Projects", "Hardware procurement with expert lifecycle advice", "24/7 Premium Technical Support"]
}

Generate ONLY the JSON list and nothing else.
"""

def generate_question_plan() -> list:
    """
    Uses Gemini to generate a dynamic and comprehensive list of questions based on its internal
    knowledge of the Business Model Canvas methodology.
    """
    ai_box("Jsem expert na metodologii Business Model Canvas. Připravuji pro vás personalizovaný plán dotazování, abychom prozkoumali váš byznys do hloubky.", title="🧠 Příprava Plánu")

    # The prompt is now self-contained, no files needed.
    response_text = ask_gemini_sdk(LLM_EXPERT_QUESTION_PLANNER, temperature=0.2)

    if "AI_ERROR" in response_text:
        ai_box("Nepodařilo se mi vytvořit plán. Zkuste prosím spustit buňku znovu.", title="❌ Chyba Plánu")
        return []

    try:
        cleaned_json_text = response_text.strip().lstrip("```json").rstrip("```").strip()
        question_plan = json.loads(cleaned_json_text)
        if isinstance(question_plan, list) and all('key' in item and 'question' in item and 'coverage_points' in item for item in question_plan):
            ai_box(f"Plán dotazování byl úspěšně vygenerován. Zeptám se vás na {len(question_plan)} klíčových oblastí.", title="✅ Plán Připraven")
            return question_plan
        else:
            raise ValueError("Vygenerovaný JSON postrádá požadované klíče.")
    except (json.JSONDecodeError, ValueError) as e:
        ai_box(f"Nastala chyba při zpracování vygenerovaného plánu: {e}.", title="❌ Chyba Zpracování")
        return []

# The conduct_dynamic_bmc_analysis function from the previous version is perfect and needs NO changes.
# We will just copy it here to keep Cell 5 self-contained.
def conduct_dynamic_bmc_analysis(question_plan: list) -> dict:
    ai_box("Nyní společně projdeme jednotlivé bloky vašeho byznys modelu do hloubky.", title="🚀 Jdeme na to!")
    bmc_data = {}
    for i, config in enumerate(question_plan):
        display_status_message(f"Oblast {i+1} z {len(question_plan)}: {config.get('key', 'Neznámý blok').replace('_', ' ').title()}")
        response = get_user_input_with_llm_validation(
            bmc_block_question=config.get('question', 'Chybí text otázky.'),
            block_name=config.get('key', f'Otázka {i+1}'),
            coverage_points=config.get('coverage_points', []),
            it_examples=config.get('examples', [])
        )
        bmc_data[config.get('key', f'custom_question_{i+1}')] = response
    ai_box("Skvělá práce! Zmapovali jsme celý váš byznys model.", title="🎉 Hotovo!")
    return bmc_data

In [ ]:
# Cell 6: Phase 3 - LLM-Powered Input Analysis
# ... (LLM_DEEP_ANALYSIS_PERSONA_V2 prompt remains the same) ...

def perform_llm_bmc_analysis(bmc_data: dict) -> str:
    # This function no longer needs the file_refs parameter
    display_status_message("Initiating expert strategic analysis...")
    bmc_data_string = "\n".join([f"- {key}: {value}" for key, value in bmc_data.items() if value != "Skipped"])
    analysis_prompt = f"{LLM_DEEP_ANALYSIS_PERSONA_V2}\n\nHere is the BMC data from the user:\n{bmc_data_string}"
    return ask_gemini_sdk(analysis_prompt, temperature=0.8)

In [ ]:
# Cell 7: Phase 4 - LLM-Powered Suggestion Generation
# ... (LLM_INNOVATION_SUGGESTION_PERSONA_V2 prompt remains the same) ...

def generate_llm_suggestions(bmc_data_str: str, analysis_summary: str) -> str:
    # This function no longer needs the file_refs parameter
    display_status_message("Generating innovation proposals based on expert analysis...")
    suggestion_prompt = f"""...""" # The prompt content is the same
    return ask_gemini_sdk(suggestion_prompt, temperature=1.2)

In [ ]:
# Cell 8: Main Execution Flow (Simplified - No File Upload)

# ... (display_llm_output helper function remains the same) ...

def run_main_session():
    """Orchestrates the entire AI-driven BMC Navigator session."""
    if not model:
        ai_box("Gemini model was not initialized. The session cannot start.", title="KRITICKÁ CHYBA")
        return

    # Phase 1: Welcome and Plan Generation
    display_welcome_and_introduction() # Shows the welcome message
    question_plan = generate_question_plan() # Immediately generates the plan

    if not question_plan:
        ai_box("Nepodařilo se mi připravit plán dotazování. Zkuste prosím spustit sezení znovu.", title="❌ Chyba Spuštění")
        return

    # Phase 2: Dynamic Questioning
    current_bmc_data = conduct_dynamic_bmc_analysis(question_plan)

    # Phase 3: Analysis
    analysis_result = perform_llm_bmc_analysis(current_bmc_data)
    display_llm_output("Fáze 3: Strategická Analýza", analysis_result)
    input("Uživatel (Stiskněte Enter pro pokračování k Návrhům Inovací): ")

    # Phase 4: Suggestions
    bmc_summary_str = "\n".join([f"- {k}: {v}" for k, v in current_bmc_data.items() if v != "Skipped"])
    suggestions_result = generate_llm_suggestions(bmc_summary_str, analysis_result)
    display_llm_output("Fáze 4: Návrhy Inovací", suggestions_result)

    # Conclusion
    ai_box(
        "Tímto končí naše interaktivní sezení s BMC Navigátorem. Analýza a návrhy byly založeny na expertní znalosti standardní Business Model Canvas metodologie.",
        title="🎉 Sezení Dokončeno!"
    )

# --- Spuštění Hlavního Sezení ---
if __name__ == "__main__":
    run_main_session()

NameError: name 'display_status_message' is not defined